# Visualization 3

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
import requests
import re

# new import statements
import geopandas as gpd
from shapely.geometry import Point, Polygon, box

In [ ]:
# Find the path for "naturalearth_lowres"
path = gpd.datasets.get_path("naturalearth_lowres")
# Read the shapefile for "naturalearth_lowres" and
# set index using "name" column
gdf = gpd.read_file(path).set_index("name")

### Extracting "Europe" data from "naturalearth_lowres"

In [ ]:
# Europe bounding box
eur_window = box(-10.67, 34.5, 31.55, 71.05)

Can we use `intersects` method?

In [ ]:
gdf.intersects(eur_window)

In [ ]:
# Incorrect
???.plot()

Can we use `intersection` method?

In [ ]:
gdf.intersection(eur_window)

In [ ]:
???.plot()

How can we get rid of empty polygons?
- Sometimes empty polygons will mess up with your plot in case of specifying colors or other formatting

In [ ]:
eur = gdf.intersection(eur_window)
eur.???

Remove all the empty polygons using `is_empty`.

In [ ]:
eur = ???
eur

In [ ]:
eur.plot()

### Centroids of European countries

In [ ]:
# plot the centroids
eur.plot(facecolor="lightgray", edgecolor="k")
eur.???

### Lat / long CRS

- Long is x-coord
- Lat is y-coord
    - tells you where the point on Earth is
- **IMPORTANT**: degrees are not a unit of distance. 1 degree of longitute near the equator is a lot farther than moving 1 degree of longitute near the north pole

Using `.crs` to access CRS of a gdf.




In [ ]:
eur.???

#### Single meter-based CRS doesn't work for the whole earth

- Setting a different CRS for Europe that is based on meters.
- https://spatialreference.org/ref/?search=europe

In [ ]:
# Setting CRS to "EPSG:3035" for European Union
eur_m = eur.???
eur_m.crs

In [ ]:
ax = eur_m.plot(facecolor="lightgray", edgecolor="k")
eur_m.centroid.plot(ax=ax)

#### How much error does lat/long computation introduce?

In [ ]:
ax = eur_m.plot(facecolor="lightgray", edgecolor="k")
eur_m.centroid.plot(ax=ax, ???) # black => correct
eur.centroid.???.plot(ax=ax, ???)  # red => miscalculated

### Areas of European countries

In [ ]:
eur_m.??? # area in sq meters

What is the area in **sq miles**?

In [ ]:
# Conversion: / 1000 / 1000 / 2.59
(eur_m.area / ???).???
# careful!  some countries (e.g., Russia) were cropped when we did intersection

In [ ]:
# area on screen, not real area
eur.area

## Madison area emergency services

- Data source: https://data-cityofmadison.opendata.arcgis.com/
    - Search for:
        - "City limit"
        - "Lakes and rivers"
        - "Fire stations"
        - "Police stations"

- CRS for Madison area: https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Universal_Transverse_Mercator_zones.svg

In [ ]:
city = gpd.read_file("City_Limit.zip").to_crs(???)
# 3rd digit (6 stands for north hemisphere, 7 stands for south)
# last two digits stand for the vertical mercator coordinate 

In [ ]:
city.crs

In [ ]:
water = gpd.read_file("Lakes_and_Rivers.zip").to_crs(???)
fire = gpd.read_file("Fire_Stations.zip").to_crs(???)
police = gpd.read_file("Police_Stations.zip").to_crs(???)

#### Run this on your virtual machine

`sudo sh -c "echo 'Options = UnsafeLegacyRenegotiation' >> /etc/ssl/openssl.cnf"`

then restart notebook!

#### GeoJSON

How to find the below URL?

- Go to info page of a dataset, for example: https://data-cityofmadison.opendata.arcgis.com/datasets/police-stations/explore?location=43.081769%2C-89.391550%2C12.81
- Then click on "I want to use this" > "View API Resources" > "GeoJSON"

In [ ]:
url = "https://maps.cityofmadison.com/arcgis/rest/services/Public/OPEN_DATA/MapServer/2/query?outFields=*&where=1%3D1&f=geojson"
police2 = gpd.read_file(url).to_crs(city.crs)

In [ ]:
ax = city.plot(color="lightgray")
water.plot(color="lightblue", ax=ax)
fire.plot(color="red", ax=ax, ???)
police2.plot(color="blue", ax=ax, ???)
ax.???
ax.???

In [ ]:
fire.to_file("fire.geojson")

### Geocoding: street address => lat / lon

- Daily incident reports: https://www.cityofmadison.com/fire/daily-reports

In [ ]:
url = "https://www.cityofmadison.com/fire/daily-reports"
r = requests.get(url)
r

In [ ]:
r.raise_for_status() # give me an exception if not 200 (e.g., 404 file not found)

In [ ]:
# doesn't work
# pd.read_html(url)

In [ ]:
# print(r.text)

Find all **span** tags with **streeAddress** using regex.

In [ ]:
addrs = re.findall(r'<span itemprop="streetAddress">(.*?)</span>', r.text)[:-1]
addrs = pd.Series(addrs) + "; Madison, WI"
addrs

In [ ]:
gpd.tools.geocode("4200 block West Beltline Highway; Madison, WI")

In [ ]:
incidents = gpd.tools.geocode(addrs).dropna()
incidents

In [ ]:
ax = city.plot(color="lightgray")
water.plot(color="lightblue", ax=ax)
fire.plot(color="red", ax=ax, marker="+", label="Fire")
police2.plot(color="blue", ax=ax, marker="^", label="Police")
incidents.to_crs(city.crs).plot(ax=ax, color="k", marker="*", label="Incidents")
ax.legend(loc="upper left", frameon=False)
ax.set_axis_off()